In [1]:
from EMAN3 import *
from EMAN3tensor import *
import numpy as np
tf_set_device(dev=0,maxmem=4096)

2024-04-24 22:37:07.189668: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-24 22:37:09.698021: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-24 22:37:09.801514: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#

In [2]:
orts=Orientations(np.array([[0,0,0],[0,0,.25],[0,0,.5],[0,0,.75],[0.125,0,0],[0.25,0,0],[0.375,0,0],[0.5,0,0]]))

In [3]:
orts.to_mx2d()

<tf.Tensor: shape=(2, 3, 8), dtype=float32, numpy=
array([[[ 1.0000000e+00, -1.1920929e-07, -1.0000000e+00, -1.1920929e-07,
          1.0000000e+00,  1.0000000e+00,  1.0000000e+00,  1.0000000e+00],
        [ 0.0000000e+00, -1.0000000e+00,  8.7422777e-08,  1.0000000e+00,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00]],

       [[ 0.0000000e+00,  1.0000000e+00, -8.7422777e-08, -1.0000000e+00,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 1.0000000e+00, -1.1920929e-07, -1.0000000e+00, -1.1920929e-07,
          7.0710671e-01, -1.1920929e-07, -7.0710695e-01, -1.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         -7.0710683e-01, -1.0000000e+00, -7.0710683e-01,  8.7422777e-08]]],
      dtype=float32)>

In [3]:
gaus=Gaussians(np.array([[0.0,0.0,0.0,1.0],[.3,0,0,1],[0,.3,0,.5],[0,0,.3,.25],[0,0,-.1,.25]]))

In [5]:
imgs=gaus.project_simple(orts,64)

2024-02-03 20:23:07.132239: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [6]:
emimgs=from_tf(imgs,True)

In [7]:
imgsf=tf_fft2d(imgs)

In [10]:
tf_frc(imgsf[:2],imgsf[2:4],True)

(46,) (64, 33, 1) (2, 64, 33)
(46,) (64, 33, 1) (2, 64, 33)


<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.7433562, 0.7350201], dtype=float32)>

In [ ]:
ptcls=EMData.read_images("/home2/stevel/test3d/ptcls_08_even.hdf")
xfs=[ptcl["xform_projection"] for ptcl in ptcls]

In [3]:
type(np.zeros(5,dtype="float32"))

numpy.ndarray

In [2]:
N=8
x=EMStack2D([test_image(size=(1024,1024)) for i in range(N)])
for i in range(N): x.emdata[i].translate(i*3,i*2,0)

In [4]:
xf=x.do_fft()
cf=xf.calc_ccf(xf[0],center=True)
#cfp=tf_phaseorigin2d(cf.tensor)
cfr=cf.do_ift()
EMData.write_images("ccf.hdf",cfr.emdata)

True

In [4]:
tf.complex64

tf.complex64

In [5]:
x[0].div

<tf.Tensor: shape=(1024, 1024), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [6]:
# Developing init_from_map method
emd=test_image_3d()
seg=emd.process("segment.gauss",{"minratio":0.1,"width":4,"skipseg":1})
amps=np.array(seg["segment_amps"])
centers=np.array(seg["segment_centers"]).reshape(len(amps),3)
centers/=(emd["nx"],emd["ny"],emd["nz"])
centers-=(0.5,0.5,0.5)
amps/=max(amps)
print(centers.shape,amps.shape)
cc=np.concatenate((centers.transpose(),amps.reshape((1,len(amps)))))
print(cc.shape,cc[0])
#

(168, 3) (168,)
(4, 168) [ 0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.4375
 -0.4375    -0.375      0.375     -0.34375    0.3359375 -0.3125
 -0.4921875  0.4921875  0.296875  -0.28125    0.2578125 -0.25
 -0.2109375  0.2109375 -0.1796875  0.171875  -0.1484375  0.1328125
 -0.1171875  0.09375   -0.0859375 -0.0546875  0.0546875  0.
  0.         0.         0.         0.         0.        -0.40625
  0.40625    0.         0.         0.         0.         0.
  0.        -0.4609375  0.46875    0.         0.         0.03125
  0.         0.         0.234375  -0.0390625  0.

In [2]:
# Testing orientation conversion and making sure projection directions match

map1=EMData("A.hdf")
gaus=Gaussians()
gaus.init_from_map(map1,4,.5)
print(len(gaus)," gaussians init")

ptcls=EMStack2D("A_proj.hdf")
orts,txty=ptcls.orientations

projs=gaus.project_simple(orts,map1["nx"],txty=txty)
#projs.write_images("A_reproj.hdf")
projsf=projs.do_fft()
ptclsf=ptcls.do_fft()


524  gaussians init


2024-04-25 20:11:34.788336: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [18]:
frcs=tf.stack(tf_frc(projsf.tensor,ptclsf.tensor,0))
tf.math.reduce_mean(frcs[:,:10],1)

In [28]:
#Initialize Gaussians to random values
rnd=tf.random.uniform((100,4))
rnd+=(-.5,-.5,-.5,100.0)
gaus._data=rnd/(4.0,4.0,4.0,100.0)



In [31]:
gaus[:10],gaus[-10:]

(<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
 array([[-0.06804289, -0.01572398,  0.13215725,  0.9901045 ],
        [ 0.07442839, -0.08245361, -0.1251184 ,  1.0649745 ],
        [ 0.06811549, -0.05047636, -0.10591442,  0.9965065 ],
        [-0.05886522, -0.06504864,  0.02711095,  0.94404334],
        [-0.09623458,  0.11744577,  0.03807257,  0.93351775],
        [ 0.1380825 , -0.05167935, -0.08641298,  1.0887544 ],
        [-0.09520481,  0.0150267 , -0.06673578,  1.0326577 ],
        [-0.01770332, -0.0244867 , -0.10012727,  0.9975222 ],
        [-0.03816146, -0.05606935, -0.13237397,  1.0952895 ],
        [-0.07896011, -0.06265385, -0.0904939 ,  1.036947  ]],
       dtype=float32)>,
 <tf.Tensor: shape=(10, 4), dtype=float32, numpy=
 array([[-0.08951983, -0.11614624, -0.08334197,  0.95720875],
        [ 0.0622078 ,  0.00565374,  0.06604336,  1.0348835 ],
        [ 0.02437813, -0.02123026,  0.0367891 ,  1.0026112 ],
        [ 0.00796423,  0.02169406, -0.10173687,  1.0567319 ],
       

In [12]:
ptclsfds=ptcls.downsample(24)    # downsample specifies the final size, not the amount of downsampling
ny=ptclsfds.shape[1]

In [24]:
max(gaus[:,3])

<tf.Tensor: shape=(), dtype=float32, numpy=1.1404998>

In [34]:
# "learn" Gaussian locations test on downsampled particles
for i in range(32):
    with tf.GradientTape() as gt:
        gt.watch(gaus.tensor)
        projs=gaus.project_simple(orts,ny,txty=txty)
        projsf=projs.do_fft()
        frcs=tf_frc(projsf.tensor,ptclsfds.tensor,-1)
    grad=gt.gradient(frcs,gaus._data)
    qual=tf.math.reduce_mean(frcs)
    shift=tf.math.reduce_std(grad[:,:3])
    sca=tf.math.reduce_std(grad[:,3])
    xyzs=1.0/(shift*1000)   # xyz scale
    gaus.add_tensor(grad*(xyzs,xyzs,xyzs,1.0/(sca*500)))
    print(f"{float(qual)}\t{float(shift)}\t{float(sca)}")


0.4218832552433014	6.183226108551025	0.36069416999816895
0.42969566583633423	7.907054424285889	0.3653753101825714
0.43414178490638733	6.972530841827393	0.36840447783470154
0.44206348061561584	5.775845527648926	0.3556816577911377
0.44925758242607117	5.793899059295654	0.3658798933029175
0.4553244709968567	6.702793598175049	0.3627201020717621
0.4589085280895233	5.514772891998291	0.36930277943611145
0.46441978216171265	4.699159622192383	0.3559567630290985
0.4689394533634186	5.045386791229248	0.3653113842010498
0.47201672196388245	6.286284446716309	0.3570215404033661
0.4753434658050537	5.394957542419434	0.36661234498023987
0.47919219732284546	5.8590569496154785	0.3523448705673218
0.48179927468299866	5.974534034729004	0.367990106344223
0.4855543375015259	5.692536354064941	0.3534810245037079
0.48825570940971375	6.254665851593018	0.36589059233665466
0.4912266731262207	6.365291595458984	0.3560218811035156
0.49370893836021423	5.9564290046691895	0.3614468276500702
0.4967828392982483	5.61642122268

In [61]:
# "learn" Gaussian locations test
nx=map1["nx"]
for i in range(16):
    with tf.GradientTape() as gt:
        gt.watch(gaus.tensor)
        projs=gaus.project_simple(orts,nx,txty=txty)
        projsf=projs.do_fft()
        frcs=tf_frc(projsf.tensor,ptclsf.tensor,28)
    grad=gt.gradient(frcs,gaus._data)
    qual=tf.math.reduce_mean(frcs)
    shift=tf.math.reduce_std(grad[:,:3])
    xyzs=1.0/(shift*1000)   # xyz scale
    gaus.add_tensor(grad*(xyzs,xyzs,xyzs,0.0))
    print(f"{float(qual)}\t{float(shift)}")


0.5992359519004822	3.607722520828247
0.6222471594810486	2.9173660278320312
0.6405577659606934	2.299583673477173
0.6548011302947998	1.8245238065719604
0.6660016179084778	1.4766881465911865
0.675089418888092	1.2581884860992432
0.682397723197937	1.0515044927597046
0.6877304315567017	0.993125855922699
0.6915033459663391	1.104539155960083
0.6943872570991516	1.1641767024993896
0.6964638829231262	1.1463381052017212
0.6974629759788513	1.2180379629135132
0.6984103322029114	1.2186729907989502
0.6990590691566467	1.237385869026184
0.6998651623725891	1.2380579710006714
0.7003649473190308	1.2703198194503784


In [10]:
projs.write_images("A_reproj_opt.hdf")

In [33]:
vol=gaus.volume(map1["nx"])
vol.write_images("A_vol_opt.hdf")

In [19]:
t=Transform()
tuple(t.get_trans_2d())

(0.0, 0.0)

In [18]:
np.array(((1,1),(0,0),(1,0))).shape

(3, 2)